### Установка пакетов

In [1]:
!pip install pyspark

### Импорт библиотек

In [2]:
import json
import os
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql import SparkSession
from sklearn.datasets import make_classification

### Создание каталога data для хранения данных

In [3]:
if not os.path.exists('data'):
  os.makedirs('data')

### Создание экземпляра SparkSession

In [4]:
spark = SparkSession.builder.appName('Homework_3_lab3').getOrCreate()

## Бинарная классификация (BinaryClassification)

### Задание 1. Сгенерировать подходящие исходные данные для проведения обучения

#### 1.1. Определение функции генерации данных

In [5]:
def to_file(path, X, y):
  lines = []

  for i in range(len(y)):
    features = []

    for j in range(len(X[0])):
      features.append(f"{j+1}:{X[i][j]}")

    lines.append(f"{y[i]} {' '.join(features)}")

  with open(path, "w") as file:
    file.write('\n'.join(lines))

#### 1.2. Генерация данных

In [6]:
X, y = make_classification(
  n_samples=1000,
  n_features=15,
  n_informative=12,
  n_redundant=3,
  n_clusters_per_class=2
)

to_file('data/classification_data', X, y)

#### 1.3. Чтение данных из файла

In [7]:
df = spark.read.format('libsvm').load('data/classification_data')
df.head()

Row(label=1.0, features=SparseVector(15, {0: -2.5421, 1: -0.2669, 2: -0.1468, 3: -0.2975, 4: 2.617, 5: 0.1723, 6: -3.35, 7: 3.2991, 8: -0.7271, 9: 0.7871, 10: -0.4746, 11: 1.3936, 12: -0.2733, 13: 6.2807, 14: 1.3179}))

### Задание 2. Провести бинарную классификацию с помощью pyspark.ml / pyspark.mllib

In [8]:
df_prepared = df.rdd.map(lambda row: LabeledPoint(row.label, list(row.features))).collect()
df_prepared = spark.sparkContext.parallelize(df_prepared)

In [9]:
df_train, df_test = df_prepared.randomSplit([0.8, 0.2])

In [10]:
model = LogisticRegressionWithLBFGS.train(df_train)

In [11]:
predictions = df_test.map(lambda row: (float(model.predict(row.features)), row.label))

### Задание 3. Провести расчет метрик

In [12]:
metrics = BinaryClassificationMetrics(predictions)
(metrics.areaUnderPR, metrics.areaUnderROC)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


(0.7746040723981901, 0.7974472807991121)

### Задание 4. Сохранить полученные значения метрик в файл

In [13]:
metrics_list = [ metrics.areaUnderPR, metrics.areaUnderROC ]

with open('data/classification_data_r_1', 'w') as file:
  file.write(str(metrics_list))